In [35]:
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix
from scipy.stats.mstats import gmean
from sklearn.model_selection import StratifiedKFold, train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

fpath = "/Users/ys8mz/Box Sync/Predictive Models of College Completion (VCCS)/intermediate_files"
results_dir = "C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\smaller_training_sample\\test1\\"

In [36]:
df = pd.read_stata(fpath + "/full_data_truncated_survival.dta")
df = df.iloc[:,[-3,-2,-1] + list(range(df.shape[1]-3))]
for p in ['num_terms', 'times', 'event']:
    df.loc[:,p] = df[p].astype(int)

In [37]:
train_df_old = df[df.valid == 0].drop(['valid'], axis=1)
test_df = df[df.valid == 1].drop(['valid'], axis=1)
print(train_df_old.shape,test_df.shape)

(298139, 334) (33115, 334)


##### Randomly select 10% observations from the original training sample

In [38]:
_, train_df = train_test_split(train_df_old, test_size=0.1, stratify=train_df_old['event'].astype(str)+"_"+train_df_old['num_terms'].astype(str), random_state=54321)

In [39]:
predictors = list(df.columns)[4:]
print(len(predictors))
impute_list_1 = set(["prop_comp_pre","cum_gpa_pre"])
impute_list_2 = set([t1+"_"+t2+str(t3) for t1 in ["term_gpa", "prop_comp", "lvl2_prop_comp", "dev_prop_comp"] for t2 in ["fa", "sp", "su"] for t3 in range(1,7,1)])
impute_list_3 = set(["cum_gpa", "lvl2_prop_comp", "dev_prop_comp", "prop_comp", "prop_comp_sd", "withdrawn_prop_comp_sd"])
impute_list_4 = set(["admrate", "gradrate", "satvr25", "satvr75", "satmt25", "satmt75", "satwr25", "satwr75"])

331


In [40]:
def impute(train, test):
    for p in impute_list_1:
        avg_p = np.nanmean(train[train.enrolled_pre == 1][p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    for p in impute_list_3:
        avg_p = np.nanmean(train[p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    for p in impute_list_2:
        suffix = p[-3:]
        avg_p = np.nanmean(train[train["enrolled_" + suffix] == 1][p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    for p in impute_list_4:
        avg_p = np.nanmean(train[train["enrolled_nsc"] == 1][p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    return train, test   

In [41]:
%time train_df_new, test_df_new = impute(train_df, test_df)

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Wall time: 24.3 s


In [42]:
y_test = test_df_new.event.copy()
test_df_new.loc[:,'event'] = 0

In [12]:
# Fix the CoxPH model
cph = CoxPHFitter(penalizer=0.01) # No penalizer will result in LinAlgError (probably due to multicollinearity)
%time cph.fit(train_df_new, duration_col='times', event_col='event', strata='num_terms')

C:\Users\ys8mz\AppData\Roaming\Python\Python35\site-packages\lifelines\utils\__init__.py:1086: ConvergenceWarning: Column(s) ['enrl_intensity_nsc_fa6', 'enrl_intensity_nsc_sp6', 'enrl_intensity_nsc_su6'] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


Wall time: 42.1 s


<lifelines.CoxPHFitter: fitted with 29814 total observations, 19618 right-censored observations>

In [13]:
# Predict graduation probability (derived from survival probability by the end of Year 6)
test_pred_raw = cph.predict_survival_function(test_df_new).T.sort_index()
print(test_pred_raw.shape)
y_test_pred = [1 - test_pred_raw.iloc[i,:].loc[18-e] for i,e in enumerate(test_df_new.num_terms)]

(33115, 17)


In [1]:
pickle.dump(list(y_test_pred), open(results_dir + "/y_test_pred_cox.p", "wb"))

In [14]:
# C-statistics
print("Cox Proportional Hazard Model:")
print("AUC = {}".format(round(roc_auc_score(y_test, y_test_pred),4)))

Cox Proportional Hazard Model:
AUC = 0.8768


In [15]:
def find_optimal_threshold(p,r,t):
    to_drop = np.union1d(np.where(pd.isnull(p[:-1]) == True)[0], np.where(pd.isnull(r[:-1]) == True)[0])
    to_drop = np.union1d(to_drop, np.where(pd.isnull(t) == True)[0])
    to_keep = np.setdiff1d(np.array(list(range(len(p)-1))), to_drop)
    p,r,t = p[to_keep],r[to_keep],t[to_keep]
    f1 = 2*p*r/(p+r)
    best_t = t[np.argmax(f1)]
    best_t
    return best_t

In [16]:
def cross_validation_cox(train):
    threshold_list = []
    auc_list = []
    k_fold =  StratifiedKFold(n_splits = 10, random_state = 12345, shuffle=True)
    for train_indices, test_indices in k_fold.split(train, train.event):
        train_part = train.iloc[train_indices,:]
        test_part = train.iloc[test_indices,:]
        train_part_new, test_part_new = impute(train_part, test_part)
        test_part_new = test_part_new.sort_index()
        y_2 = test_part_new.event.copy()
        test_part_new.loc[:,'event'] = 0
        model = CoxPHFitter(penalizer=0.01)
        model.fit(train_part_new, duration_col='times', event_col='event', strata='num_terms', step_size=0.1)
        y_2_pred_raw = model.predict_survival_function(test_part_new).T.sort_index()
        y_2_pred = [1 - y_2_pred_raw.iloc[i,:].loc[18-e] for i,e in enumerate(test_part_new.num_terms)]
        p,r,t = precision_recall_curve(y_2, y_2_pred)
        auc = roc_auc_score(y_2, y_2_pred)
        threshold_list.append(find_optimal_threshold(p,r,t))
        auc_list.append(auc)
    print(threshold_list)
    print(np.mean(auc_list), np.std(auc_list, ddof=1))
    return gmean(threshold_list)   

In [17]:
best_threshold = cross_validation_cox(train_df)

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\ys8mz\AppData\Roaming\Python\Python35\site-packages\lifelines\utils\__init__.py:1086: ConvergenceWarning: Column(s) ['enrl_intensity_nsc_fa6', 'enrl_intensity_nsc_sp6', 'enrl_intensity_nsc_su6'] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

[0.36573440262788026, 0.9999998110637521, 0.37452322342084243, 0.9999993403295645, 0.3674965741569305, 0.3831245466594647, 0.29618604660136516, 0.38987885960821156, 0.357951078368111, 0.39965386489115184]
0.8750848589640448 0.008005561486421077


In [18]:
best_threshold

0.44701690212755923

In [19]:
def create_confusion_matrix(y_test_pred, threshold, fname):
    cm_arr = confusion_matrix(y_test, np.where(y_test_pred > threshold, 1, 0))
    cm_df = pd.DataFrame(cm_arr, columns=['Pred_0','Pred_1'], index=['Real_0', 'Real_1'])
    cm_df.loc[:,''] = cm_df.sum(axis=1)
    cm_df.loc['',:] = cm_df.sum(axis=0)
    print(cm_df)
    print("")
    p1 = cm_df.iloc[1,1]/cm_df.iloc[2,1]
    r1 = cm_df.iloc[1,1]/cm_df.iloc[1,2]
    p0 = cm_df.iloc[0,0]/cm_df.iloc[2,0]
    r0 = cm_df.iloc[0,0]/cm_df.iloc[0,2]    
    print("F1 score = {}".format(round(2*p1*r1/(p1+r1),4)))    
    cm_df.to_csv(results_dir + fname + ".csv")
    return p1,r1,p0,r0,round(2*p1*r1/(p1+r1),4)

In [20]:
print("F1 threshold = {}:\n".format(str(round(best_threshold,3))))
pr_cox = create_confusion_matrix(y_test_pred, best_threshold, "Cox_cm1")

F1 threshold = 0.447:

         Pred_0   Pred_1         
Real_0  19420.0   2388.0  21808.0
Real_1   3587.0   7720.0  11307.0
        23007.0  10108.0  33115.0

F1 score = 0.721


In [21]:
num_of_0 = int(round((1-np.mean(train_df.event))*len(y_test)))
y_test_pred_binary = np.ones(len(y_test))
y_test_pred_binary[np.argsort(y_test_pred)[:num_of_0]] = 0
alternative_threshold = y_test_pred[np.argsort(y_test_pred)[num_of_0]]
print("Alternative threshold = {}:\n".format(str(round(alternative_threshold,3))))
pr2_cox = create_confusion_matrix(y_test_pred_binary, best_threshold, "Cox_cm2")

Alternative threshold = 0.397:

         Pred_0   Pred_1         
Real_0  18765.0   3043.0  21808.0
Real_1   3025.0   8282.0  11307.0
        21790.0  11325.0  33115.0

F1 score = 0.7319


In [22]:
precision_recall_df = pd.DataFrame([(best_threshold,)+pr_cox,(alternative_threshold,)+pr2_cox]).round(4)
precision_recall_df.index = ['F1','Same_Graduation_Rate']
precision_recall_df.columns = ['threshold','precision_1','recall_1','precision_0','recall_0','f1_score']
precision_recall_df.to_csv(results_dir + "Cox_precision_recall.csv", index=True)

#### Comparison with reduced validation

##### Randomly select 10% observations from the original validation sample

In [56]:
test_df_new.loc[:,'event'] = list(y_test)
_, test_df_reduced = train_test_split(test_df_new, test_size=0.1, stratify=test_df_new['event'].astype(str)+"_"+test_df_new['num_terms'].astype(str), random_state=54321)
test_df_reduced = test_df_reduced.sort_index()

In [57]:
y_test_reduced = np.array(test_df_reduced.event)
test_df_reduced.loc[:,'event'] = 0

In [58]:
results_dir_new1 = "C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\smaller_training_sample\\test1\\comparison_1\\"

In [59]:
print("Cox Proportional Hazard:")
test_pred_raw = cph.predict_survival_function(test_df_reduced).T.sort_index()
print(test_pred_raw.shape)
y_test_pred = [1 - test_pred_raw.iloc[i,:].loc[18-e] for i,e in enumerate(test_df_reduced.num_terms)]
print("AUC = {}".format(round(roc_auc_score(y_test_reduced, y_test_pred),4)))

Cox Proportional Hazard:
(3312, 17)
AUC = 0.8866


In [60]:
pickle.dump(list(y_test_pred), open(results_dir_new1 + "y_test_pred_cox.p", "wb"))

In [61]:
def create_confusion_matrix_new(y_test, y_test_pred, threshold, fpath, fname):
    cm_arr = confusion_matrix(y_test, np.where(np.array(y_test_pred) > threshold, 1, 0))
    cm_df = pd.DataFrame(cm_arr, columns=['Pred_0','Pred_1'], index=['Real_0', 'Real_1'])
    cm_df.loc[:,''] = cm_df.sum(axis=1)
    cm_df.loc['',:] = cm_df.sum(axis=0)
    print(cm_df)
    print("")
    p1 = cm_df.iloc[1,1]/cm_df.iloc[2,1]
    r1 = cm_df.iloc[1,1]/cm_df.iloc[1,2]
    p0 = cm_df.iloc[0,0]/cm_df.iloc[2,0]
    r0 = cm_df.iloc[0,0]/cm_df.iloc[0,2]    
    print("F1 score = {}".format(round(2*p1*r1/(p1+r1),4)))    
    cm_df.to_csv(fpath + fname + ".csv")
    return p1,r1,p0,r0,round(2*p1*r1/(p1+r1),4)

In [64]:
def create_pr(train_df, y_test, y_test_pred, best_threshold, fpath, mn):
    print("F1 threshold = {}:\n".format(str(round(best_threshold,3))))
    pr_lr = create_confusion_matrix_new(y_test, y_test_pred, best_threshold, fpath, "{}_cm1".format(mn))

    num_of_0 = int(round((1-np.mean(train_df.event))*len(y_test)))
    y_test_pred_binary = np.ones(len(y_test))
    y_test_pred_binary[np.argsort(y_test_pred)[:num_of_0]] = 0
    alternative_threshold = y_test_pred[np.argsort(y_test_pred)[num_of_0]]
    print("\n\n")
    print("Alternative threshold = {}:\n".format(str(round(alternative_threshold,3))))
    pr2_lr = create_confusion_matrix_new(y_test, y_test_pred_binary, best_threshold, fpath, "{}_cm2".format(mn))

    precision_recall_df = pd.DataFrame([(best_threshold,)+pr_lr,(alternative_threshold,)+pr2_lr]).round(4)
    precision_recall_df.index = ['F1','Same_Graduation_Rate']
    precision_recall_df.columns = ['threshold','precision_1','recall_1','precision_0','recall_0','f1_score']
    precision_recall_df.to_csv(fpath + "{}_precision_recall.csv".format(mn), index=True)

In [65]:
create_pr(train_df, y_test_reduced, y_test_pred, best_threshold, results_dir_new1, "Cox")

F1 threshold = 0.447:

        Pred_0  Pred_1        
Real_0  1934.0   247.0  2181.0
Real_1   329.0   802.0  1131.0
        2263.0  1049.0  3312.0

F1 score = 0.7358



Alternative threshold = 0.404:

        Pred_0  Pred_1        
Real_0  1894.0   287.0  2181.0
Real_1   285.0   846.0  1131.0
        2179.0  1133.0  3312.0

F1 score = 0.7473


#### Comparison with the base model (trained on the full training data), using the reduced validation sample

In [66]:
model_dir = "C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\truncated_predictors\\"
with open(model_dir + 'cox.pickle', 'rb') as f:
    cph = pickle.load(f) # saving my trained cph model as my.pickle

In [68]:
train_df_old = df[df.valid == 0]
test_df_old = df[df.valid == 1]
_, test_df_new = train_test_split(test_df_old, test_size=0.1, stratify=test_df_old['event'].astype(str)+"_"+test_df_old['num_terms'].astype(str), random_state=54321)
_, test_df_reduced = impute(train_df_old, test_df_new)
test_df_reduced = test_df_reduced.sort_index()
y_test_reduced = np.array(test_df_reduced.event)
test_df_reduced.loc[:,'event'] = 0

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [69]:
results_dir_new2 = "C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\smaller_training_sample\\test1\\comparison_2\\"

In [70]:
print("Cox Proportional Hazard:")
test_pred_raw = cph.predict_survival_function(test_df_reduced).T.sort_index()
print(test_pred_raw.shape)
y_test_pred = [1 - test_pred_raw.iloc[i,:].loc[18-e] for i,e in enumerate(test_df_reduced.num_terms)]
print("AUC = {}".format(round(roc_auc_score(y_test_reduced, y_test_pred),4)))

Cox Proportional Hazard:
(3312, 17)
AUC = 0.8901


In [71]:
pickle.dump(list(y_test_pred), open(results_dir_new2 + "y_test_pred_cox.p", "wb"))

In [72]:
create_pr(train_df_old, y_test_reduced, y_test_pred, best_threshold, results_dir_new2, "Cox")

F1 threshold = 0.447:

        Pred_0  Pred_1        
Real_0  1934.0   247.0  2181.0
Real_1   323.0   808.0  1131.0
        2257.0  1055.0  3312.0

F1 score = 0.7392



Alternative threshold = 0.414:

        Pred_0  Pred_1        
Real_0  1893.0   288.0  2181.0
Real_1   286.0   845.0  1131.0
        2179.0  1133.0  3312.0

F1 score = 0.7465
